In [ ]:
import pandas as pd

df = pd.read_csv('turnbackhoax(fix).csv')
df.head()

,ID,judul,isi,url
0,1,700 Kepala Desa Ditangkap KPK,Akun Facebook “Trie Sudirman” pada Rabu (4/6/2...,https://turnbackhoax.id/2025/06/13/salah-700-k...
1,2,Pesan Berantai Proses Pembuatan Beras dari Bot...,Pada Minggu (1/6/2025) beredar sebuah pesan be...,https://turnbackhoax.id/2025/06/12/salah-pesan...
2,3,Penegasan Nadiem Makarim Berbagi Uang Pengadaa...,Pada Minggu (1/5/2025) beredar sebuah unggahan...,https://turnbackhoax.id/2025/06/12/salah-peneg...
3,4,Sertifikat Halal Produk McDonald’s Dicabut di ...,\nAkun Tiktok “aliimranmaryam” pada Senin (7/1...,https://turnbackhoax.id/2025/06/11/salah-serti...
4,5,Tautan Pendaftaran “Rekrutmen Program Literasi...,Pada Sabtu (24/5/2025) akun Facebook “Andika S...,https://turnbackhoax.id/2025/06/09/penipuan-ta...


In [ ]:
import openai
import time
import os
import pandas as pd
import os
from tqdm import tqdm # Import tqdm untuk progress bar

# --- Konfigurasi ---
openai.api_key = "" # YOUR API KEY

# --- PROMPT TERBAIK (FEW-SHOT) ---
# Mendefinisikan prompt sebagai konstanta agar kode lebih bersih.
# Prompt ini telah dirancang untuk menangani berbagai format data Anda.
PROMPT_TEMPLATE = """
Anda adalah AI yang sangat ahli dalam ekstraksi data. Tugas utama Anda adalah mengekstrak teks narasi hoax dari artikel periksa fakta.

Instruksi Penting:
1.  **Ekstrak Hanya Teks Hoax:** Salin kata per kata hanya teks hoax-nya.
2.  **Abaikan Bagian Lain:** Abaikan SEMUA bagian lain, termasuk judul, ringkasan, nama penulis, kategori, sumber, dan terutama seluruh blok "PENJELASAN" dan "REFERENSI".
3.  **Jangan Sertakan Label:** Jangan sertakan label seperti "NARASI:" atau frasa pengantar seperti "terdapat narasi:" dalam hasil akhir.
4.  **Gabungkan Narasi Terpisah:** Jika narasi hoax terbagi menjadi beberapa bagian (misalnya, ada sebelum dan sesudah blok "PENJELASAN"), Anda HARUS menemukan dan menggabungkan semua bagian tersebut menjadi satu teks yang utuh.

Berikut adalah beberapa contoh untuk memandu Anda:

---
**Contoh 1: Kasus dengan label "NARASI:" yang jelas**
**Input:**
"
Hasil periksa fakta Rahmah an. KATEGORI: IMPOSTER CONTENT/KONTEN TIRUAN... NARASI: “Untuk tenaga honorer yang ikut tes CPNS bisa langsung hubungi saya...” PENJELASAN: Beredar akun Facebook...
"
**Output yang Diharapkan:**
"Untuk tenaga honorer yang ikut tes CPNS bisa langsung hubungi saya untuk pendaftaran ke BKN akan dapat bantu lolos ASN 2023 lengkapi persyaratan untuk tenaga honorer Kabupaten Paser"

---
**Contoh 2: Kasus tanpa label "NARASI:" dan narasi berada dalam kutipan**
**Input:**
"
BUKAN tentara Cina melainkan pasukan Brimob... Akun Facebook Imho Walleye mengunggah sebuah video dengan narasi: “SAMPAI SEKARANG BELUM ADA SEORANG PEJABAT INDONESIA YANG BISA MENJELAS SIAPA ORANG-ORANG INI...” dan “APAKAH VIDEO INI BENAR ???...” PENJELASAN: Berdasarkan hasil penelusuran...
"
**Output yang Diharapkan:**
"SAMPAI SEKARANG BELUM ADA SEORANG PEJABAT INDONESIA YANG BISA MENJELAS SIAPA ORANG-ORANG INI... dan APAKAH VIDEO INI BENAR ???..."

---
**Contoh 3: Kasus dengan narasi terpisah sebelum dan sesudah "PENJELASAN"**
**Input:**
"
[SALAH] Video amatir detik-detik sebelum kejadian. Narasi awal menyebutkan bahwa peristiwa ini terjadi karena kelalaian petugas di lapangan. PENJELASAN: Faktanya, setelah ditelusuri... Narasi lanjutan yang beredar di grup pesan juga menambahkan klaim bahwa “sudah ada 10 korban jiwa akibat insiden ini”.
"
**Output yang Diharapkan:**
"Narasi awal menyebutkan bahwa peristiwa ini terjadi karena kelalaian petugas di lapangan. Narasi lanjutan yang beredar di grup pesan juga menambahkan klaim bahwa “sudah ada 10 korban jiwa akibat insiden ini”."

---
**Tugas Anda Sekarang**
**Input:**
"{text_input}"

**Output yang Diharapkan:**
"""

def ekstrak_narasi_hoax(teks):
    """
    Mengirim teks ke OpenAI API untuk mengekstrak narasi hoax
    berdasarkan template prompt yang telah ditentukan.
    """
    # Menggunakan prompt template yang sudah kita definisikan
    prompt = PROMPT_TEMPLATE.format(text_input=teks)

    try:
        response = openai.chat.completions.create(
            # Menggunakan model yang lebih efisien dan direkomendasikan
            model="o4-mini-2025-04-16",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=512,  # Alokasi token yang cukup untuk narasi
            temperature=0.1, # Temperatur sangat rendah untuk hasil yang konsisten
        )

        # Pengecekan respons untuk memastikan tidak ada error
        if response.choices and response.choices[0].message.content:
            return response.choices[0].message.content.strip()
        else:
            print("Warning: Menerima respons kosong dari API.")
            return "" # Kembalikan string kosong jika tidak ada hasil

    except Exception as e:
        print(f"Terjadi error saat memproses teks: {e}")
        # Kembalikan string kosong agar mudah difilter nanti jika terjadi error
        return ""

# --- Proses DataFrame dengan Loop dan Progress Bar (Lebih Skalabel) ---
hasil_bersih = []
# Gunakan tqdm untuk melihat progress bar. Ganti df['isi'] dengan df['isi'].head(5) untuk testing
for artikel in tqdm(df['isi_berita'], desc="Mengekstrak Narasi"):
    hasil = ekstrak_narasi_hoax(str(artikel))
    hasil_bersih.append(hasil)
    time.sleep(1) # Jeda 1 detik untuk menghindari rate limit API

# Tambahkan hasil ke DataFrame
df['narasi_hoax'] = hasil_bersih

# Tampilkan hasil
print("\nHasil Ekstraksi:")
print(df[['isi_berita', 'narasi_hoax']])